# 概要
このノートブックは、Vertex AIプロンプトオプティマイザーを紹介します。これは、ターゲット固有のメトリクスを使用して、ターゲットモデル（例：`gemini-2.5-flash`）に適合するようにプロンプトを反復的に最適化するツールです。

主なユースケース：

* プロンプト最適化：ターゲットモデルの最適な入力特性に合わせて構造と内容を改良することで、初期プロンプトの品質を向上させます。
* プロンプト翻訳：あるモデル用に最適化されたプロンプトを、別のターゲットモデルで効果的に動作するように適応させます。

詳細なドキュメントについては[こちら](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer)をご覧ください。

![How to use vapo](https://raw.githubusercontent.com/GoogleCloudPlatform/gcp-getting-started-lab-jp/master/minitap/images/vapo.png)

## 事前準備

### ノートブック環境の認証（Colabのみ）

Google Colab上で環境を認証します。

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### サンプルデータ生成

In [ ]:
!pip install --upgrade google-genai --user
!gcloud auth application-default login

In [ ]:
from google import genai
from google.genai import types

def generate(item):
  client = genai.Client(
      vertexai=True,
      project="project-id",
      location="global",
  )

  si_text1 = """あなたは家電量販店で家電を購入した顧客です。購入した商品に対するレビューを書いてください。
ただし、メーカー名、ブランド名などの固有名詞は避けてください。
最後にレビューを200文字程度に要約してください。"""

  model = "gemini-2.5-pro"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=f"""購入品: {item}
review:
summary:""")
      ]
    ),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    seed = 0,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = "application/json",
    response_schema = {"type":"ARRAY","items":{"type":"OBJECT","properties":{"review":{"type":"STRING"},"summary":{"type":"STRING"}}}},
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )

  output = ""
  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")
    output += chunk.text
  return output

#generate()

In [ ]:
import json

# Items for reviews
items = ["冷蔵庫","洗濯機","掃除機","電子レンジ","テレビ"]

# Generate full review and summary for each items
reviews = []
for item in items:
    review = generate(item)
    reviews.append(json.loads(review)[0])

In [ ]:
# Write jsonl file
with open('reviews.jsonl', mode='w', encoding='utf-8') as f:
    for obj in reviews:
        json.dump(obj, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
!gcloud storage cp ./reviews.jsonl gs://bucket-id

# ステップ0：パッケージとライブラリのインストール

In [3]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py
import vapo_lib

--2025-08-04 01:05:35--  https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37206 (36K) [text/plain]
Saving to: ‘vapo_lib.py’

vapo_lib.py         100%[===================>]  36.33K  --.-KB/s    in 0.02s   

2025-08-04 01:05:35 (2.13 MB/s) - ‘vapo_lib.py’ saved [37206/37206]



# ステップ1：プロンプトテンプレートとシステム指示の作成
以下にシステム指示とプロンプトテンプレートを入力してください。手順については[こちら](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#template-si)を参照してください。

プロンプトは2つの主要なコンポーネントで構成されています：

- システム指示：システム指示は、プロンプト内のユーザー入力の前にモデルに渡される指示です。これは、特定のタスクのすべてのクエリで共有されるプロンプトテンプレートの固定部分です。
- プロンプトテンプレート：タスクは、モデルに応答を提供してもらいたいプロンプト内のテキストです。コンテキストは、モデルが応答を生成する際に使用または参照する情報で、プロンプトに含めます。これらは、タスクに基づいて変化するプロンプトテンプレートの動的な部分です。

プロンプトオプティマイザーは、システム指示テンプレートの最適化または翻訳を可能にしますが、プロンプトテンプレートは最適なシステム指示テンプレートを評価および選択するために不可欠です。

In [4]:
#SYSTEM_INSTRUCTION = (
#    "Answer the following question. Let's think step by step.\n"  # @param
#)
#PROMPT_TEMPLATE = "Question: {question}\n\nAnswer: {target}"  # @param

#SYSTEM_INSTRUCTION = (
#    "Based on the following images and articles respond to the questions.'\n' Be concise, and answer \"I don't know\" if the response cannot be found in the provided articles or images."
#)
SYSTEM_INSTRUCTION = (
    "タスク:\n以下のレビューを簡単にいうと?"
)

#PROMPT_TEMPLATE = "Article 1:\n\n{article_1}\n\nImage 1:\n\n{image_1} @@@image/jpeg\n\nQuestion: {question}"
PROMPT_TEMPLATE = "レビューテキスト:\n{review}\n\n 要点:\n{summary}"

# ステップ2：プロジェクト設定の構成
ターゲットのGoogleモデル用にプロンプトを最適化するには、ラベル付きの検証サンプル（入力と正解出力のペア）を含むCSVまたはJSONLファイルを提供してください。手順については[こちら](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#prepare-sample-prompts)を参照してください。

対処したい問題を具体的に示す例に焦点を当ててください。
推奨事項：信頼性の高い結果を得るには、50〜100個の異なるサンプルを使用してください。ただし、最小5つのサンプルでも効果的な場合があります。
プロンプト翻訳の場合（例：サードパーティモデルからGoogleモデル、PaLM 2からGemini）：

ターゲットモデルが苦手とする例にソースモデルを使用してラベルを付けることを検討し、改善すべき領域を特定するのに役立ててください。
ソースモデルを選択する場合、入力例のラベルを提供する必要はありません。
ソースモデルの選択はGoogleモデルに限定されていますが、Google以外のモデルからのラベル付き入力もサポートしています。Google以外のソースモデルを選択したい場合は、入力例のラベルを提供する必要があります。

In [ ]:
# @markdown **プロジェクト設定**: <br/>
PROJECT_ID = "project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
OUTPUT_PATH = "gs://bucket-id/output"  # @param {type:"string"}
INPUT_DATA_PATH = "gs://bucket-id/reviews.jsonl"  # @param {type:"string"}

# ステップ3：最適化設定の構成
最適化設定は、最も一般的に使用される値にデフォルト設定されており、初期設定として使用することをお勧めします。

最も重要な設定は以下の通りです：

* ターゲットモデル：プロンプトを最適化しようとしているモデル
* 最適化モード：プロンプトを最適化しようとしているモード
* 評価メトリクス：プロンプトを最適化しようとしている評価メトリクス
* 翻訳ソースフィールド名：CometやMetricXなどの翻訳メトリクスが選択されている場合は、データ内のソーステキストの対応するフィールド名を入力してください。それ以外の場合は空のままにしてください。

すべての評価メトリクスは、値が大きいほど良いという性質を持つことが期待されることに注意してください。そのため、MetricXの値を0（最悪）から25（最良）の間に変更しました。
さまざまな設定とその最適な活用方法の詳細については、[こちら](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#configuration)を参照してください。

In [8]:
TARGET_MODEL = "gemini-2.5-flash-lite"  # @param ["gemini-2.5-flash", "gemini-2.5-flash-lite"]
OPTIMIZATION_MODE = "instruction_and_demo"  # @param ["instruction", "demonstration", "instruction_and_demo"]
EVAL_METRIC = "summarization_quality"  # @param ["bleu", "coherence", "comet", "exact_match", "fluency", "groundedness", "metricx", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
TRANSLATION_SOURCE_FIELD_NAME = ""  # @param {type:"string"}

# ステップ4：高度な最適化設定の構成 [オプション]
さまざまな設定とその最適な活用方法の詳細については、[こちら](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#configuration)を参照してください。

In [ ]:
# @markdown **指示最適化設定**: <br/>
NUM_INST_OPTIMIZATION_STEPS = 10  # @param {type:"integer"}

# @markdown **デモンストレーション最適化設定**: <br/>
NUM_DEMO_OPTIMIZATION_STEPS = 10  # @param {type:"integer"}
NUM_DEMO_PER_PROMPT = 3  # @param {type:"integer"}

# @markdown **モデル設定**: <br/>
TARGET_MODEL_QPS = 3.0  # @param {type:"number"}
EVAL_QPS = 3.0  # @param {type:"number"}

# @markdown **マルチメトリクス設定**: <br/>
# @markdown 最適化に複数のメトリクスが必要な場合のみ、このセクションを使用してください。これにより、上記で選択したメトリクスが上書きされます。
EVAL_METRIC_1 = "NA"  # @param ["NA", "bleu", "coherence", "comet", "exact_match", "fluency", "groundedness", "metricx", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_1_WEIGHT = 0.0  # @param {type:"number"}
EVAL_METRIC_2 = "NA"  # @param ["NA", "bleu", "coherence", "comet", "exact_match", "fluency", "groundedness", "metricx", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_2_WEIGHT = 0.0  # @param {type:"number"}
EVAL_METRIC_3 = "NA"  # @param ["NA", "bleu", "coherence", "comet", "exact_match", "fluency", "groundedness", "metricx", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_3_WEIGHT = 0.0  # @param {type:"number"}
METRIC_AGGREGATION_TYPE = "weighted_sum"  # @param ["weighted_sum", "weighted_average"]

# @markdown **その他の設定**: <br/>
PLACEHOLDER_TO_VALUE = "{}"  # @param
RESPONSE_MIME_TYPE = "text/plain"  # @param ["text/plain", "application/json", "text/x.enum"] {type:"string"}
RESPONSE_SCHEMA = ""
TARGET_LANGUAGE = "Japanese"  # @param ["English", "French", "German", "Hebrew", "Hindi", "Italian", "Japanese", "Korean", "Portuguese", "Simplified Chinese", "Spanish", "Traditional Chinese"] {type:"string"}
TOOLS = ""  # @param
TOOL_CONFIG = ""  # @param

# ステップ5：プロンプトオプティマイザーの実行
ジョブの所要時間を知らせるプログレスバーが表示されます。

In [14]:
import datetime
import json

timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
display_name = f"pt_{timestamp}"

label_enforced = vapo_lib.is_run_target_required(
    [
        EVAL_METRIC,
        EVAL_METRIC_1,
        EVAL_METRIC_2,
        EVAL_METRIC_3,
    ],
    "",
)
input_data_path = f"{INPUT_DATA_PATH}"
vapo_lib.validate_prompt_and_data(
    "\n".join([SYSTEM_INSTRUCTION, PROMPT_TEMPLATE]),
    input_data_path,
    PLACEHOLDER_TO_VALUE,
    label_enforced,
)

output_path = f"{OUTPUT_PATH}/{display_name}"

params = {
    "project": PROJECT_ID,
    "num_steps": NUM_INST_OPTIMIZATION_STEPS,
    "system_instruction": SYSTEM_INSTRUCTION,
    "prompt_template": PROMPT_TEMPLATE,
    "target_model": TARGET_MODEL,
    "target_model_qps": TARGET_MODEL_QPS,
    "target_model_location": LOCATION,
    "eval_qps": EVAL_QPS,
    "optimization_mode": OPTIMIZATION_MODE,
    "num_demo_set_candidates": NUM_DEMO_OPTIMIZATION_STEPS,
    "demo_set_size": NUM_DEMO_PER_PROMPT,
    "aggregation_type": METRIC_AGGREGATION_TYPE,
    "data_limit": 50,
    "input_data_path": input_data_path,
    "output_path": output_path,
    "response_mime_type": RESPONSE_MIME_TYPE,
    "response_schema": RESPONSE_SCHEMA,
    "language": TARGET_LANGUAGE,
    "placeholder_to_content": json.loads(PLACEHOLDER_TO_VALUE),
    "tools": TOOLS,
    "tool_config": TOOL_CONFIG,
    "translation_source_field_name": TRANSLATION_SOURCE_FIELD_NAME,
}

if EVAL_METRIC_1 == "NA":
    params["eval_metrics_types"] = [EVAL_METRIC]
    params["eval_metrics_weights"] = [1.0]
else:
    metrics = []
    weights = []
    for metric, weight in zip(
        [EVAL_METRIC_1, EVAL_METRIC_2, EVAL_METRIC_3],
        [EVAL_METRIC_1_WEIGHT, EVAL_METRIC_2_WEIGHT, EVAL_METRIC_3_WEIGHT],
    ):
        if metric == "NA":
            break
        metrics.append(metric)
        weights.append(weight)
    params["eval_metrics_types"] = metrics
    params["eval_metrics_weights"] = weights

job = vapo_lib.run_apd(params, OUTPUT_PATH, display_name)
print(f"Job ID: {job.name}")

progress_form = vapo_lib.ProgressForm(params)
while progress_form.monitor_progress(job):
    pass # Add a pass statement to ensure the while loop has a body
    # time.sleep(5)



Job display name: pt_2025-08-04T01:20:56
Job ID: 8116682333316186112




Instruction Optimization


IntProgress(value=0, description='Progress', max=10)


Generated Templates:


step,prompt,metrics.summarization_quality/mean
0,タスク:\n以下のレビューを簡単にいうと?,4.8
1,タスク:\n以下のレビューの主要な内容と特徴を、重要な詳細を省略せずに簡潔に要約してください。,5.0
1,タスク:\n以下のレビューを、提供されている「要点」の形式と詳細度を参考にしつつ、簡潔に要約すると？,5.0
2,タスク:\n提供された「レビューテキスト」の内容のみに基づいて簡潔に要約してください。クエリ内に示されている「要点」は、出力する要約の形式、スタイル、詳細度の参考としてのみ使用し、その内容を要約の対象としないでください。,5.0
2,タスク:\n以下のレビューテキストを要約してください。入力には「レビューテキスト:」と、その要約例である「要点:」が含まれます。「要点:」は、要約の形式、簡潔さ、重要な詳細の含め方の参考として利用してください。\nあなたの要約は、「レビューテキスト:」の内容のみに基づき、主要な内容と特徴を、重要な詳細を省略せずに簡潔にまとめる必要があります。\n最終的な要約は「The response is:」の後に記述してください。,4.8
3,タスク:\n入力には「レビューテキスト」と、そのレビューに対する理想的な要約例である「要点」が含まれています。\n「レビューテキスト」を読み、その製品やサービスの主要な利点、特徴、使用感、そしてもしあれば課題点を抽出してください。\n「要点」の形式と簡潔さを参考にしながら、これらの重要な情報を漏らすことなく、読者が内容を素早く理解できるよう、簡潔かつ分かりやすく要約を生成してください。,4.6
3,タスク:\n以下のレビューを要約してください。その際、入力に含まれる「要点」の例が示す**形式**（文章構造、句読点、表現スタイルなど）と**詳細度**（情報量、粒度）を**厳密に模倣し、完全に一致する**ように要約を作成してください。,5.0
4,タスク:\n以下のレビューを、提供されている「要点」の例に示されている、見出しを含まない形式と詳細度を厳密に踏襲し、簡潔に要約すると？,4.6
4,タスク:\n以下のレビューテキストから、製品の主要な特徴、それらがもたらす具体的な利点、および使用によって得られる体験や状況といった重要な詳細を省略せずに、簡潔に要約してください。提供される「要点」は、要約のスタイルと内容の粒度を理解するための参考としてください。,4.6
5,タスク:\n入力に含まれる「要点」セクションに記載されているテキストを、見出し（例:「要点:」）を付けずに、その内容、形式、詳細度を完全に維持したまま、正確に再現して出力してください。,1.8


...


Generated Templates:


step,prompt,metrics.summarization_quality/mean
0,タスク:\n以下のレビューの主要な内容と特徴を、重要な詳細を省略せずに簡潔に要約してください。,5.0
1,タスク:\n以下のレビューの主要な内容と特徴を、重要な詳細を省略せずに簡潔に要約してください。\nレビューテキスト:\n今回新しく購入した家電は、日々の食卓を支える上で欠かせない存在となっています。以前使っていたものが古くなったため買い替えましたが、その進化に驚いています。特に、直感的に操作できるパネルは誰にでも使いやすく、迷うことがありません。食品の温めムラがなく、冷凍品も均一に解凍・加熱してくれるので、調理の失敗が減りました。庫内は見た目以上に広く、様々なサイズの食器に対応できるのも便利です。自動センサーでの温め機能も非常に優秀で、食材の種類や量に応じて最適な仕上がりにしてくれます。デザインもシンプルで、キッチンの雰囲気を損なわずに設置できました。毎日の食事の準備が格段にスムーズになり、この家電が来てから生活の質が上がったと感じています。本当に購入して良かったです。\n\n 要点:\n以前の加熱調理器からの買い替えで、今回手にした新製品に大変満足しています。直感的な操作性に加え、食品の温めムラがなく、冷凍品も均一に加熱できる点が素晴らしいです。庫内が広く、自動センサーであたためも完璧。毎日の食事準備が格段に楽になり、シンプルでキッチンに馴染むデザインも気に入っています。家事の負担が軽減され、生活の質が向上したと実感しています。購入して本当に良かったです。\n\nThe response is: このレビューは、買い替えた新しい家電製品への満足を述べています。以前の製品からの進化に驚き、特に操作パネルの直感的な使いやすさを評価しています。食品の温めムラのなさ、冷凍品の均一な解凍・加熱、庫内の広さ、自動センサーによる最適な仕上がりなど、具体的な機能面での利便性を挙げています。また、デザインがシンプルでキッチンに馴染む点も評価しています。これらの要素により、日々の食事準備がスムーズになり、生活の質が向上したと感じていることが述べられています。\n\n==\n\n\nレビューテキスト:\n先日、新しい洗濯機を購入しました。まず驚いたのが、その洗浄力の高さです。諦めていたガンコな汚れもきちんと落ちて、衣類が本当にきれいになります。運転中の音も非常に静かで、夜間に使用しても全く気になりません。以前のモデルと比べて節水効果も高いようで、水道代の節約にも貢献してくれそうです。操作パネルも直感的で、初めて使う時でも迷うことなく設定できました。デザインもシンプルで、洗面所の雰囲気を損なわないのが嬉しいです。大家族なので容量の大きさが必須でしたが、一度にたくさんの洗濯物を処理できるので、洗濯の回数を減らせて非常に助かっています。日々の家事が格段に楽になり、大変満足しています。\n\n 要点:\n先日購入した洗濯機は、高い洗浄力でガンコな汚れもすっきり。運転音も静かで夜間使用も安心です。節水効果も高く、家計に優しい点も気に入っています。直感的な操作パネルとシンプルなデザインも魅力。大容量なので洗濯回数を減らせ、日々の家事が劇的に楽になり大変満足しています。\n\nThe response is: 先日購入した洗濯機は、**高い洗浄力で諦めていたガンコな汚れも落とす**。**運転音は非常に静かで夜間でも気にならない**。**節水効果が高く水道代の節約に貢献**。**操作パネルは直感的で使いやすく、デザインはシンプルで洗面所の雰囲気を損なわない**。**大容量で一度に多くの洗濯物を処理でき、洗濯回数を減らせる**ため、日々の家事が楽になり大変満足している。\n\n==\n\n\nレビューテキスト:\nこの度、新しい掃除機を購入しました。まず、その吸引力には大変驚いています。絨毯の奥に潜んだゴミも、フローリングの細かいホコリも一度でしっかりと吸い取ってくれます。以前使っていたものと比べても格段に性能が上がったと感じます。本体は非常に軽く、階段の持ち運びも楽で、コードレスなので部屋の隅々までストレスなく掃除ができます。操作性もシンプルで、誰でもすぐに使いこなせると思います。稼働音も想像していたよりもずっと静かで、夜間でも気にせず使用できる点が助かっています。付属のアタッチメントも豊富で、家具の隙間やカーテンのホコリ取り、車の掃除など、様々な用途に活躍してくれます。バッテリーの持ちも十分で、一度の充電で家全体を掃除しきるのに困ることはありません。デザインもスタイリッシュで、部屋に置いてあっても違和感がありません。全体的に見て、非常に満足度の高い買い物でした。毎日のお掃除が楽しくなりました。\n\n 要点:\n購入した新しい掃除機は、その圧倒的な吸引力に驚きました。軽量でコードレスのため、家中どこでも手軽に掃除でき、操作も簡単です。作動音も静かで、付属のアタッチメントが豊富なので、様々な場所で活躍します。バッテリーも長持ちし、デザインも良く、毎日の掃除が快適になりました。大変満足しています。\n\nThe response is: 購入した掃除機は、**吸引力が非常に高く、絨毯の奥のゴミやフローリングの細かいホコリも一度で除去できる**点が特筆される。**軽量かつコードレス**であるため、階段の持ち運びや部屋の隅々までの掃除が容易。**操作はシンプルで、稼働音も静か**。**豊富なアタッチメント**により、家具の隙間やカーテン、車の掃除など多用途に利用可能。**バッテリーの持ちも十分**で、一度の充電で家全体を掃除できる。**デザインもスタイリッシュ**で、全体的に満足度が高く、日々の掃除が快適になったとのこと。\n,5.0
2,タスク:\n以下のレビューの主要な内容と特徴を、重要な詳細を省略せずに簡潔に要約してください。\nレビューテキスト:\nこの大型テレビは、期待以上の高画質で、映像は非常に鮮明かつ色彩豊かです。音もクリアで迫力があり、映画やスポーツ観戦が格段に楽しくなりました。操作も直感的で、スマート機能も充実しており、ストレスなく使えています。デザインもスタイリッシュで、リビングに自然に馴染み、大変満足しています。総合的に見て、購入して本当に良かったです。\n\n 要点:\n期待以上の高画質と迫力ある音質で、映画やスポーツ観戦が格段に楽しくなる大型テレビです。直感的な操作性と充実したスマート機能も使いやすく、スタイリッシュなデザインでリビングにも馴染みます。総合的に非常に満足しており、良い買い物ができました。\n\nThe response is: この大型テレビは、高画質で映像が鮮明かつ色彩豊かであり、クリアで迫力のある音質で映画やスポーツ観戦が楽しめる。操作は直感的でスマート機能も充実しており、デザインもスタイリッシュでリビングに馴染むため、全体的に非常に満足できる買い物だった。\n\n==\n\n\nレビューテキスト:\nこの度、新しい掃除機を購入しました。まず、その吸引力には大変驚いています。絨毯の奥に潜んだゴミも、フローリングの細かいホコリも一度でしっかりと吸い取ってくれます。以前使っていたものと比べても格段に性能が上がったと感じます。本体は非常に軽く、階段の持ち運びも楽で、コードレスなので部屋の隅々までストレスなく掃除ができます。操作性もシンプルで、誰でもすぐに使いこなせると思います。稼働音も想像していたよりもずっと静かで、夜間でも気にせず使用できる点が助かっています。付属のアタッチメントも豊富で、家具の隙間やカーテンのホコリ取り、車の掃除など、様々な用途に活躍してくれます。バッテリーの持ちも十分で、一度の充電で家全体を掃除しきるのに困ることはありません。デザインもスタイリッシュで、部屋に置いてあっても違和感がありません。全体的に見て、非常に満足度の高い買い物でした。毎日のお掃除が楽しくなりました。\n\n 要点:\n購入した新しい掃除機は、その圧倒的な吸引力に驚きました。軽量でコードレスのため、家中どこでも手軽に掃除でき、操作も簡単です。作動音も静かで、付属のアタッチメントが豊富なので、様々な場所で活躍します。バッテリーも長持ちし、デザインも良く、毎日の掃除が快適になりました。大変満足しています。\n\nThe response is: 購入した掃除機は、**吸引力が非常に高く、絨毯の奥のゴミやフローリングの細かいホコリも一度で除去できる**点が特筆される。**軽量かつコードレス**であるため、階段の持ち運びや部屋の隅々までの掃除が容易。**操作はシンプルで、稼働音も静か**。**豊富なアタッチメント**により、家具の隙間やカーテン、車の掃除など多用途に利用可能。**バッテリーの持ちも十分**で、一度の充電で家全体を掃除できる。**デザインもスタイリッシュ**で、全体的に満足度が高く、日々の掃除が快適になったとのこと。\n\n==\n\n\nレビューテキスト:\n新しい冷蔵庫が届き、早速使い始めました。以前使っていたものよりも格段に広くなり、食材の整理がとてもしやすくなりました。特に野菜室の鮮度保持機能には驚いています。野菜が本当に長持ちするようになりました。また、動作音が非常に静かで、リビングにいても全く気になりません。ドアポケットもたっぷりと収納でき、使い勝手が素晴らしいです。省エネ性能も高く、電気代の削減にも期待しています。家族みんなが満足していま


Best Template so far:


Textarea(value='NA', layout=Layout(height='150px', width='80%'))

Label(value='Score: NA Improvement: NA')

# ステップ6：結果の確認
最適化プロセス中に各プロンプトテンプレートによって生成された具体的な応答をより明確に確認するには、下のセルを使用してください。
これにより、1つまたは複数のVertex Prompt Optimizer実行中にすべての生成されたテンプレートによって行われたすべての予測を検査できます。

In [ ]:
from IPython.display import HTML, display

RESULT_PATH = "gs://bucket-id/output"

results_ui = vapo_lib.ResultsUI(RESULT_PATH)

results_df_html = """
<style>
  .scrollable {
    width: 100%;
    height: 80px;
    overflow-y: auto;
    overflow-x: hidden;  /* 水平スクロールバーを非表示 */
  }
  tr:nth-child(odd) {
    background: var(--colab-highlighted-surface-color);
  }
  tr:nth-child(even) {
    background-color: var(--colab-primary-surface-color);
  }
  th {
    background-color: var(--colab-highlighted-surface-color);
  }
</style>
"""

display(HTML(results_df_html))
display(results_ui.get_container())

# [オプション]：Prompt ClassによるPromptのバージョン管理

In [ ]:
import vertexai
from vertexai.preview import prompts
from vertexai.preview.prompts import Prompt

# from vertexai.generative_models import GenerationConfig, SafetySetting # Optional

# Initialize vertexai
vertexai.init(project=PROJECT_ID, location="us-central1")

# Create local Prompt
local_prompt = Prompt(
    prompt_name="movie-critic",
    prompt_data="Compare the movies {movie1} and {movie2}.",
    variables=[
        {"movie1": "The Lion King", "movie2": "Frozen"},
        {"movie1": "Inception", "movie2": "Interstellar"},
    ],
    model_name="gemini-2.0-flash-001",
    system_instruction="You are a movie critic. Answer in a short sentence.",
    # generation_config=GenerationConfig, # Optional,
    # safety_settings=SafetySetting, # Optional,
)

# Generate content using the assembled prompt for each variable set.
for i in range(len(local_prompt.variables)):
    response = local_prompt.generate_content(
        contents=local_prompt.assemble_contents(**local_prompt.variables[i])
    )
    print(response)

# Save a version
prompt1 = prompts.create_version(prompt=local_prompt)

print(prompt1)

# Example response
# Assembled prompt replacing: 1 instances of variable movie1, 1 instances of variable movie2
# Assembled prompt replacing: 1 instances of variable movie1, 1 instances of variable movie2
# Created prompt resource with id 12345678910.....

In [ ]:
from vertexai.preview import prompts

# Get prompt
prompt = prompts.get(prompt_id="123456789")

In [ ]:
from vertexai.preview import prompts

# Restore to prompt version id 1 (original)
prompt_version_metadata = prompts.restore_version(prompt_id="123456789", version_id="1")

# Fetch the newly restored latest version of the prompt
prompt1 = prompts.get(prompt_id=prompt_version_metadata.prompt_id)

In [ ]:
from vertexai.preview import prompts

prompts.delete(prompt_id="123456789")